In [1]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import joblib
import gc

In [2]:
!ls source

82172_remove_common32_test_prob.jb model1_val.csv
82172_remove_common32_val_prob.jb  model2_test.csv
82463_test_pred_prob.csv           model2_val.csv
82463_val_pred_prob.csv            val_real_label.jb
model1_test.csv


In [4]:
t=pd.read_csv('source/model1_val.csv',header=None)
val_prob1 = t[1].values
t=pd.read_csv('source/model2_val.csv',header=None)
val_prob2 = t[1].values

del t
gc.collect()

34

In [7]:
val_x = np.array(list(zip(val_prob1, val_prob2)))
val_label = joblib.load('source/val_real_label.jb')

In [8]:
lr = LinearRegression()
lr.fit(val_x, val_label)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [9]:
lr.coef_

array([0.48391238, 0.52959348])

In [10]:
val_merge_prob = lr.coef_[0] * val_prob1 + lr.coef_[1] * val_prob2

In [11]:
#获取阈值
from sklearn.metrics import f1_score
print('=============================================== threshold search ===============================================')
# f1阈值敏感，所以对阈值做一个简单的迭代搜索
t0 = 0.05
v = 0.002
best_t = t0
best_f1 = 0

#为了快一点，缩小了范围，历史经验是在160+的时候最优
for step in range(160,191):
#for step in range(140,250,3):
    curr_t = t0 + step * v
    y = [1 if x >= curr_t else 0 for x in val_merge_prob]
    curr_f1 = f1_score(val_label, y)
    
    if curr_f1 > best_f1:
        best_t = curr_t
        best_f1 = curr_f1
        print('step: {}   best threshold: {}   best f1: {}   mean: {}'.format(step, best_t, best_f1, np.mean(y)))
print('search finish.')

=============================================== threshold search ===============================================
step: 160   best threshold: 0.37   best f1: 0.8141228344691326   mean: 0.11077613072302961
step: 161   best threshold: 0.372   best f1: 0.8141997612955038   mean: 0.11054344385159681
step: 162   best threshold: 0.374   best f1: 0.8142772220687977   mean: 0.11031934694067293
step: 163   best threshold: 0.376   best f1: 0.8143304771554577   mean: 0.11010232411487404
step: 164   best threshold: 0.378   best f1: 0.8143443785955341   mean: 0.10987216370241445
step: 165   best threshold: 0.38   best f1: 0.8144696945685822   mean: 0.10964048741457094
step: 166   best threshold: 0.382   best f1: 0.814502321438434   mean: 0.10941411669057118
step: 167   best threshold: 0.384   best f1: 0.8145147441100017   mean: 0.10920922086784371
step: 168   best threshold: 0.386   best f1: 0.8145409589968505   mean: 0.10899573508460732
step: 169   best threshold: 0.388   best f1: 0.814599259358786

In [12]:
ratio = np.mean([1 if x >= best_t else 0 for x in val_merge_prob])
print(ratio)

0.10877568050804058


In [14]:
test_prob1 = pd.read_csv('source/model1_test.csv')
test_prob1 = test_prob1['target'].values

In [15]:
#test_prob2 = joblib.load('source/82172_remove_common32_test_prob.jb')
test_prob2 = pd.read_csv('source/model2_test.csv')
test_prob2 = test_prob2['target'].values

In [16]:
test_merged_prob = lr.coef_[0] * test_prob1 + lr.coef_[1] * test_prob2

In [17]:
test_merged_prob

array([0.00137589, 0.09207078, 0.00026769, ..., 0.00016628, 0.00031076,
       0.00105396])

In [18]:
#取验证集最好的阈值
test_y = [1 if x >= best_t else 0 for x in test_merged_prob]

In [23]:
#取验证集最好的阈值 对应的百分比
threshold = sorted(test_merged_prob)[int((1-ratio) * len(test_merged_prob))]
test_y = [1 if x >= threshold else 0 for x in test_merged_prob]

print(threshold)

0.404742224908895


In [24]:
np.mean(test_y)

0.10877569252395998

In [25]:
test_num = len(test_y)
print(test_num)

3653592


In [26]:
ids = []
for i in range(test_num):
    ids.append(f'test_{i+1}')
sub = pd.DataFrame({'id': ids, 'target': test_y})

In [27]:
sub.to_csv('sub/merge_8246_0.483_8244_0.529_threshold_0.404.csv', index=False)

In [ ]:
#取算术平均
test_merged_prob = (test_prob1 + test_prob2)/2
ratio = 0.109
threshold = sorted(test_merged_prob)[int((1-ratio) * len(test_merged_prob))]
test_y = [1 if x >= threshold else 0 for x in test_merged_prob]
print(threshold)